# PCA Guided exercise

### Bioinformatics course @ University of Minho, Braga, January 2023
In this notebook we use some country-level statistics from the COVID-19 pandemic to explore the uses of PCA. The data is pre-processed and tailored to highlight some of the most relevant ideas.

Questions & comments: leonardo.garma@gmail.com

# Intro

## Load libraries & data

In [ ]:
# First we load the required libraries
import pandas as pd # Pandas handles dataframes
import seaborn as sns # Seaborn is a convenient library to plot data from dataframes
import numpy as np # Numpy has plenty of useful math functions
import matplotlib.pyplot as plt # Pyplot is a basic plotting library for Python

In [ ]:
# Read the data from the provided file. 
# OBS!!! Include the full path to the .csv file if this notebook and the .csv file are in different folders!
df=pd.read_csv('covid_processed.csv',index_col=0)

# IF you are on a Windows computer and you just downloaded the data it would look like this: 
# df=pd.read_csv('C:/Users/leona/Downloads/covid_processed.csv',index_col=0)

## Explore the data

In [ ]:
# Have a look at the data, see how big it is and look at the first lines
print(df.shape)
print(len(df))
df.head(3)


In [ ]:
# You can SLICE the data to look at specific columns/rows
df['Test_pc'] # Single column by name
df[['Test_pc','Deaths']] # Multiple columns
df.iloc[0] # Single row
df.iloc[0:5] # Multiple rows
df.iloc[0:5][['Test_pc','Deaths']] # Multiple rows and columns

In [ ]:
# You can use CONDITIONS to SLICE the data (i.e. pick rows where a condition is met):
# > larger than
# < smaller than
# == equal
# != not equal
df.loc[ df['Population']>0.5] # Rows with "Population" above 0.5 
df.loc[ df['Population']>0.5, ['GDP_pc','Recovery_rate']] # Same but picking only two columns

## Exercise 1
How many countries have a GDP per capita (GDP_pc) above 0.9? Slice the dataset to show only the Population and Life Expectancy (Life exp) of these countries

In [ ]:
## Write your answer here
#
#
#

## Plot the data

In [ ]:
# Let's try basic plotting functions from Seaborn
plt.rcParams.update({'font.size': 10}) # Make the fonts a bit larger
sns.barplot(data=df, x='Region',y='Population') # You do barplots using categories

In [ ]:
sns.catplot(data=df, x='Region',y='Population',kind='strip') # Catplot can plot categorical data in a number of ways

In [ ]:
sns.catplot(data=df, x='Region',y='Population',kind='box') # Catplot can plot categorical data in a number of ways

In [ ]:
sns.scatterplot(data=df, x='GDP_pc',y='Population', hue='Life exp') # You can use scatterplots to compare numerical variables

In [ ]:
# Using Seaborn we can also have a quick look at all the variables at once
plt.rcParams.update({'font.size': 16})
sns.pairplot(df, hue="Region")

## Exercise 2
Plot the Recovery rate (Recovery_rate) vs the number of tests per capita (Test_pc). 
Color the plot by population

In [ ]:
## Write your answer here
#
#
#

# PCA example with 2 redundant features

## Exercise 3
Create a new dataframe by Slicing the original data to keep only the "Population" and "Deaths" columns. See how these two variables relate to each other by plotting them together.

In [ ]:
# Subsample data
df2d=df[['Deaths','Population']]  #<--- Write your solution here 
sns.scatterplot() # <---- Fill in the missing parameters

## Running PCA: 
We will use the [sci-kit learn](https://scikit-learn.org) library to do PCA on the 2D data we just created.    
This library (sklearn) implements many machine learning and data analysis functions and algorithms. For PCA we will:
- Load sklearn
- Create an instance of a PCA object 
- Fit our data using this specific decomposition         

We will obtain a **principalComponents** matrix describing our data in the PC space

In [ ]:
# Run PCA with 2 components
from sklearn.decomposition import PCA # Import library
pca = PCA(n_components=2) # Instantiate the PCA object


### *Why do we use n_components=2? What happens if you use a different number?

In [ ]:
principalComponents = pca.fit_transform(df2d) # Use the PCA object to transform the data


## Exercise 4
What are the dimensions of the principalComponents matrix?

In [ ]:
## Write your answer here
#
#
#

In [ ]:
# Now we will put the projected data into a new data frame and plot it 
dfPCs = pd.DataFrame(data = principalComponents, columns = ['PC1', 'PC2']) # New dataframe
dfPCs['Region']=df['Region'].values  # Copy the region and country names from the original data
dfPCs['Country']=df.index.values

# Check the new dataframe
dfPCs.head(3)

## Exercise 5
Plot the data projected on the two PCs and color it by Country. **Notice the different scales of the two PCs!**

\* Use **plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)** to move the legend outside the plot

In [ ]:
sns.scatterplot(data=dfPCs, ... ) # <--- Fill in the blanks here
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)

### Check the explained variance of each PC
The pca object we created contains information about the decomposition. We can see how much variance is explained by each PC and how each original feature contributes to each PC.

In [ ]:
# See the variance ratios of the 2 PCs
print(pca.explained_variance_ratio_)

In [ ]:
# See the components of each PC
print(pca.components_)

### To make this a bit more visual, we can overlay the PCs on the original data...

In [ ]:
# Plot the two features and overlay the PCs scaled by their explained variance
plt.rcParams['figure.figsize']=(6,6) # Adjust the figure size
sns.scatterplot(data=df,x='Population',y='Deaths',hue='Region') # Plot the original data

scale1=pca.explained_variance_ratio_[0] # Scale the first PC by its fraction of explained variance
# Plot an arrow for the first PC
plt.arrow(x=0.1, y=0.1, dx=pca.components_[0][0]*scale1,
          dy=pca.components_[0][1]*scale1,color='r',head_width=0.05,
         label='PC1')

scale2=pca.explained_variance_ratio_[1] # Scale the second PC by its fraction of explained variance
# Plot an arrow for the second PC
plt.arrow(x=0.5, y=0.52, dx=pca.components_[1][0]*scale2,
          dy=pca.components_[1][1]*scale2,color='b',head_width=0.03,
          label='PC2')

plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0) # Place the legend outside

# PCA example with 2 pairs of redundant features

### Exercise 6
Subset the data to the columns 'Population','Deaths','GDP_pc','Deaths_per_M'. See how these features relate to each other and run PCA on it. You can follow these steps:
- Create a new dataframe with only the 4 selected columns
- Make a pairplot to visualize relationships in the data
- Instantiate a PCA object with the **right** number of dimensions
- Transform the data using the new PCA object

How many PCs do you get?
How are the explained variances distributed among the PCs? **How much variance would we retain if we use only the first 2 PCs?**


In [ ]:
# Fill in the blanks below or implement a different solution

df4d=df[[ ... ]] 

sns.pairplot( ... )

pca=PCA(n_components= ...)

principalComponents = ...


### PCA projections
Now we can see how the data looks like on the different PCs and how they relate to the original features

In [ ]:
# Make a dataframe with the projected data
dfPCs = pd.DataFrame(data = principalComponents, columns = ['PC1', 'PC2','PC3','PC4']) # New dataframe
dfPCs['Region']=df['Region'].values  # Copy the region and country names from the original data
dfPCs['Country']=df.index.values

In [ ]:
# Plot the data projected on the first two PCs
sns.scatterplot(data=dfPCs,x='PC2',y='PC1',hue='Region')
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

In [ ]:
# Plot the data projected on the first and last PCs 
# OBS!!! Notice the scales
sns.scatterplot(data=dfPCs,x='PC1',y='PC4',hue='Region')
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

#### We can use the *components_* attribute from the pca object to see the contributions of each feature to the PCs

In [ ]:
# Use the loadings from the PCA object to make a new dataframe
loadings=pd.DataFrame(pca.components_)
# Name the columns after the original features 
loadings.columns=df4d.columns.values
# Check the dataframe
loadings

Here we have one row per PC, indicating the contribution of each feature to each PC. You can visualize it easily using barplots:

In [ ]:
sns.barplot(x=loadings.columns, y=loadings.iloc[0].values) # PC1

In [ ]:
sns.barplot(x=loadings.columns, y=loadings.iloc[1].values) # PC2

# PCA with non-informative features

#### Some of the features in the dataset have little or no variance. See the distribution of the Life expectancy and the Recovery rates:
    

In [ ]:
plt.rcParams['figure.figsize']=(12,2) #rescale figures
plt.rcParams.update({'font.size': 16}) # Increase font
sns.barplot(data=df,x=df.index,y='Life exp') # Plot the Life expectancy values for each country
plt.tick_params(labelbottom=False) # Remove x-axis labels for clarity

In [ ]:
sns.barplot(data=df,x=df.index,y='Recovery_rate') # Plot the Recovery rates 
plt.tick_params(labelbottom=False) # Remove x-axis labels for clarity

## Exercise 7
What happens when we include these features on the PCA? Slice the data to include all the features 
('Population','Deaths','GDP_pc','Deaths_per_M','Life exp','Recovery_rate') and run PCA with the **appropiate**
number of components. Like on the previous exercise, you can:
    
- Create a new dataframe with the 6 features
- Instantiate a PCA object with the **right** number of dimensions
- Transform the data using the new PCA object

How many PCs do you get this time?
How are the explained variances distributed among the PCs? **How much variance are we keeping if we use the 2 first PCs?**


In [ ]:
# Fill in the blanks below or implement a different solution

df6d=df[[ ... ]] 

pca=PCA(n_components= ...)

principalComponents = ...

#### We can visualize the variance explained by each PC in a "scree plot" to decide how many we should keep

In [ ]:
plt.rcParams['figure.figsize']=(8,8) # Adjust the plot features
plt.rcParams.update({'font.size': 22})
sns.scatterplot(x=['PC1', 'PC2','PC3','PC4','PC5','PC6'],y=pca.explained_variance_ratio_,s=200) # Plot the variance values


#### Just like before, we can see the contributions of each feature to each PC. Notice which PCs relate to the Recovery rate and the Life Expectancy

In [ ]:
# Use the loadings from the PCA object to make a new dataframe
loadings=pd.DataFrame(pca.components_)
# Name the columns after the original features 
loadings.columns=df6d.columns.values
# Check the dataframe
loadings